In [1]:
import pandas as pd

In [2]:
caminho_script = 'E:/Coisas/Notas4/upperbag/'
caminho_arquivos = caminho_script + 'csvs/'
caminho_arquivos_raw = caminho_arquivos + 'raw/'
caminho_arquivos_tratados = caminho_arquivos + 'tratado/'
caminho_arquivos_vazios = caminho_arquivos + 'vazio/'

In [3]:
df_que_fem_sec = pd.read_csv( caminho_arquivos_tratados + 'quest_fem_secure.csv', encoding='utf-8')

In [4]:
#df_que_fem_sec.head()

In [5]:
def coluna_nao_aberta( nm_coluna ):
    retorno = True
    if nm_coluna.find('(outro - comente)') != -1:
        retorno = False
    if nm_coluna.find('(outros detalhes que voce nao gosta?)') != -1:
        retorno = False
    return retorno

colunas = ['Por que precisa de uma Bag'
            ,'Quais cores devemos'
            ,'Em que frequência está em cada uma das ocasiões (casual)'
            ,'Em que frequência está em cada uma das ocasiões (lazer)'
            ,'Em que frequência está em cada uma das ocasiões (sair)'
            ,'Em que frequência está em cada uma das ocasiões (social)'
            ,'Qual é o seu tipo de corpo'
            ,'Quais modelos de calçados você mais gosta'
            ,'Com qual estilo mais se identifica'
            #,'Quais categorias de itens NÃO gosta e NÃO quer receber'
            ,'Qual o fator mais importante quando você compra roupas'
            ,'Qual das combinações mais gosta'
            ,'Quais acessórios mais gostaria de receber'
            ,'Quais tipos de Calcinha você mais gosta?'
            ,'Quais categorias você gostaria de receber itens'
            ,'Quais partes do corpo gosta ou não se incomoda de mostrar/valorizar (decote_colo)'
            ,'Quais partes do corpo gosta ou não se incomoda de mostrar/valorizar (pernas)'
            ,'Quais partes do corpo gosta ou não se incomoda de mostrar/valorizar (braco)'
            ,'Quais partes do corpo gosta ou não se incomoda de mostrar/valorizar (costas)'
            ,'Quais estampas devemos EVITAR em sua bag'
            ,'Quais materiais devemos EVITAR em sua bag'
            ,'Que tipo de salto você usa'
            ,'Quais atributos NÃO gosta em calçados e NÃO quer receber'
            ,'Você se veste mais por conforto ou estilo'
            ,'Costuma encontrar algum problema de tamanho (considerando os tamanhos respondidos acima)'
            ,'Como prefere o caimento de suas blusas'
            ,'Como prefere o caimento de suas calças'
            ,'Qual altura de cintura você prefere em suas calças'
            ,'Quais as caracteristicas que mais gosta em calças'
            #,'O que você NÃO gosta em blusas'
            ,'Está disposto a experimentar diferentes itens selecionados pela sua Stylist'
            ,'Fale sobre seu estilo - você concorda'
            ,'Que tipo de alça você prefere em sutiãs']
            #,'Quais marcas você mais gosta ou mais costuma comprar roupas']
for coluna in colunas:
    df = df_que_fem_sec[coluna].str.replace(', ', ',', regex=True).str.get_dummies( sep=',' )
    for col in df.columns:
        if coluna_nao_aberta( col ): #tira comentarios
            df_que_fem_sec[coluna + '->' + col] = df[col]
            
#df_que_fem_sec = pd.get_dummies( df_que_fem_sec, columns=colunas_dummy_unico, prefix=colunas_dummy_unico, prefix_sep='->' )

In [6]:
for col in df_que_fem_sec.columns[55:]:
    print(col)

Por que precisa de uma Bag->ajuda profissional
Por que precisa de uma Bag->basicos
Por que precisa de uma Bag->calcados
Por que precisa de uma Bag->itens basico
Por que precisa de uma Bag->mais calcado
Por que precisa de uma Bag->na
Por que precisa de uma Bag->noite
Por que precisa de uma Bag->novidades
Por que precisa de uma Bag->renovar guarda
Por que precisa de uma Bag->renovar guarda roupas
Por que precisa de uma Bag->roupas para sair a noit
Por que precisa de uma Bag->roupas para trabalho informa
Por que precisa de uma Bag->surpresa
Por que precisa de uma Bag->trabalho informal
Por que precisa de uma Bag->trabalho socia
Por que precisa de uma Bag->trabalho social
Por que precisa de uma Bag->varidades e roupas para todos os estilos e ocasioe
Por que precisa de uma Bag->variedades
Por que precisa de uma Bag->variedades e roupas para todos os estilos e ocasioe
Quais cores devemos->amarelo
Quais cores devemos->azul
Quais cores devemos->branco
Quais cores devemos->cinza
Quais cores dev

In [7]:
cols = df_que_fem_sec.columns[55:]
print('Quantidade de colunas = %i' %len(cols))
data = df_que_fem_sec[ cols ].to_numpy()
print('Quantidade de linhas = %i' %len(data))

Quantidade de colunas = 328
Quantidade de linhas = 72421


In [8]:
from sklearn.cluster import KMeans

In [9]:
#kmeans = KMeans( n_clusters=4, init='random', n_init=10, max_iter=50 )
kmeans = KMeans( n_clusters=4 )
kmeans.fit( data )

KMeans(n_clusters=4)

In [10]:
df_que_fem_sec['predicao_kmeans'] = df_que_fem_sec.apply(lambda row:kmeans.predict(row[cols].to_numpy().reshape(-1,len(cols)))[0],axis=1)

In [11]:
df_que_fem_sec['predicao_kmeans'].value_counts()

0    31527
2    19294
3    12509
1     9091
Name: predicao_kmeans, dtype: int64

In [12]:
df_que_fem_sec[['id_cliente', 'predicao_kmeans']].to_csv( caminho_arquivos_tratados + 'predicoes_fem_v1.csv', index=False )
df_que_fem_sec.truncate(after=-1)[['id_cliente', 'predicao_kmeans']].to_csv( caminho_arquivos_vazios + 'predicoes_fem_v1.csv', index=False )